# Mercury data
This notebook provides the Mercury data for the simulation

Author: Sandro Ricardo De Souza: <sandro.fisica@gmail.com>

## Import modules

In [12]:
import numpy as np
import pandas as pd
import eo2pv
import math

In [13]:
# Name for each Mercury's cadidates varying the semi-axis
sufix_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

# Numbers of the Mercury's candidates varying the angles
candidates_per_dist = 100

# Conic section is ellipse
ialpha = -1

# Gravitational factor of the Sun
gm =  2.959139768995959e-04

In [14]:
# Define semi-axis
mercury_distances = np.arange(0.05, 0.55, 0.05)
a = np.repeat(mercury_distances, candidates_per_dist)

In [15]:
# Define column Mercury Name
mercury_name = []

for char in sufix_list:
    for number in range(candidates_per_dist):
        name = 'mercury' + char + str(number)
        mercury_name.append(name)

mercury_name = pd.Series(mercury_name)

In [16]:
# Define e, i
low_e = 0
high_e = 0.05 

low_i = 0
high_i = 0.05

numbers_mercurys = len(sufix_list) * candidates_per_dist

e = pd.Series(np.random.uniform(low_e, high_e, numbers_mercurys))
i = pd.Series(np.random.uniform(low_i, high_i, numbers_mercurys))

In [17]:
# Define capom omega e M
capom = np.random.uniform(0, 360, numbers_mercurys)
omega = np.random.uniform(0, 360, numbers_mercurys)
M = np.random.uniform(0, 360, numbers_mercurys)

In [18]:
# Gerando o Data Frame inicial
mercury = pd.DataFrame({'mercurys': mercury_name, 'a': a, 'e':e, 'i':i, 'capom':capom, 'omega':omega, 'M':M})
mercury = mercury.set_index('mercurys')

In [19]:
# Including mass, radio and mass_grav (equals to the Earth)
mass = 5.972300e+24
mercury['mass'] = np.repeat(mass, numbers_mercurys)
radio = 6378.137
mercury['radio'] = np.repeat(radio, numbers_mercurys)
mass_grav = 8.887539e-10
mercury['mass_grav'] = np.repeat(mass_grav, numbers_mercurys) 

In [20]:
# Including period for each mercury
def periodoKepler(semi_axis, M_planet):
    """
    Calculate the period using Kepler's third law
    Input
        semi_axis: semi-axis [au]
        M_planet: mass of the planet [kg]
    Output
        period [days]
    """
    M_sol_kg = 1.9891e30
    M_sol_G = 2.959139768995959e-04
    M_grav = M_sol_G * M_planet / M_sol_kg
    period = np.sqrt(((2 * np.pi)**2 * semi_axis**3) / (M_sol_G + M_grav))
    return(period)

In [21]:
period = periodoKepler(mercury.a, mercury.mass)

mercury['period'] = period

In [22]:
x = np.zeros(numbers_mercurys)
y = np.zeros(numbers_mercurys)
z = np.zeros(numbers_mercurys)
vx = np.zeros(numbers_mercurys)
vy = np.zeros(numbers_mercurys)
vz = np.zeros(numbers_mercurys)

for j in range(len(mercury)):
        x[j], y[j], z[j], vx[j], vy[j], vz[j] = eo2pv.orbel_el2xv(gm,ialpha,\
                                            a[j],e[j],math.radians(i[j]),\
                                            math.radians(capom[j]),math.radians(omega[j]),M[j])

In [23]:
# Create colums x, y, v, vx, vy and vz
mercury['x'] = x
mercury['y'] = y
mercury['z'] = z
mercury['vx'] = vx
mercury['vy'] = vy
mercury['vz'] = vz

In [27]:
mercury[95:105]

,M,a,capom,e,i,omega,mass,radio,mass_grav,period,x,y,z,vx,vy,vz
mercurys,,,,,,,,,,,,,,,,
mercuryA95,16.630291,0.05,327.503408,0.001260,0.005415,258.162750,5.972300e+24,6378.137,8.887539e-10,4.083678,-0.007307,0.049502,0.000004,-0.076036,-0.011302,-4.761334e-06
mercuryA96,194.150647,0.05,124.063508,0.043896,0.006419,202.899420,5.972300e+24,6378.137,8.887539e-10,4.083678,0.014820,-0.045927,0.000002,0.075126,0.026482,-8.634733e-06
mercuryA97,215.304098,0.05,13.471377,0.016152,0.037861,178.982697,5.972300e+24,6378.137,8.887539e-10,4.083678,0.017386,-0.046984,-0.000033,0.072426,0.025488,5.229916e-06
mercuryA98,170.493800,0.05,253.993189,0.046344,0.012827,205.004453,5.972300e+24,6378.137,8.887539e-10,4.083678,-0.042734,0.023000,-0.000011,-0.040025,-0.068373,-4.392067e-06
mercuryA99,163.125144,0.05,9.110025,0.006373,0.015967,155.120214,5.972300e+24,6378.137,8.887539e-10,4.083678,-0.043222,0.024515,0.000009,-0.038087,-0.067390,-1.686292e-05
mercuryB0,63.374230,0.10,256.655433,0.008348,0.032212,211.628907,5.972300e+24,6378.137,8.887539e-10,11.550385,-0.075900,0.064008,-0.000050,-0.035502,-0.041729,-1.400535e-05
mercuryB1,354.202349,0.10,246.477948,0.023079,0.002361,290.797716,5.972300e+24,6378.137,8.887539e-10,11.550385,0.069874,-0.073811,0.000004,0.039455,0.036153,8.962457e-07
mercuryB2,216.611896,0.10,47.762876,0.042669,0.018577,92.452866,5.972300e+24,6378.137,8.887539e-10,11.550385,0.069592,-0.077578,-0.000034,0.039043,0.034572,-1.837343e-06
mercuryB3,191.675243,0.10,142.371263,0.039339,0.003946,258.162776,5.972300e+24,6378.137,8.887539e-10,11.550385,-0.076561,-0.070286,0.000007,0.035424,-0.038476,6.090595e-07


In [25]:
# Create csv files from Mercury data
mercury.to_csv('Planets/mercury.csv')